In [ ]:
import os
import sys
import numpy as np
import pandas as pd
sys.path.append('../input/handm-submission-files')
from average_precision import apk
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm_notebook as tqdm
import itertools
import random
import pickle


In [ ]:
def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data

def article_id_str_to_int(series):
    return series.astype("int32")

def article_id_int_to_str(series):
    return "0" + series.astype("str")

def stringToListInt(string):
    listRes = list(string.split(" "))
    listRes = [int(s) for s in listRes]
    return listRes

def stringToList(string):
    listRes = list(string.split(" "))
    listRes = [int(s) for s in listRes]
    return listRes

def get_oof_score(answer_df):
    apks = []
    for answer, pred in zip(answer_df['answer'].values, answer_df['prediction'].values):
        answer = stringToList(answer)
        pred = stringToListInt(pred)
        apks.append(apk(answer, pred[:12]))
    score = np.mean(apks)
    #print(score)
    return score

def read_csvs_and_output_predict_columns(pahtes):
    validation_length = 68984
    predict_columns = []
    for i, path in enumerate(pathes):
        if i == 0:
            master_oof = pd.read_csv(path).sort_values('customer_id').reset_index(drop=True)[['customer_id', 'prediction']]
            master_oof.columns = [f'customer_id_sub{i}', f'prediction{i}']
            predict_columns.append(f'prediction{i}')
        else:
            sub = pd.read_csv(path).sort_values('customer_id').reset_index(drop=True)[['customer_id', 'prediction']]
            sub.columns = [f'customer_id_sub{i}', f'prediction{i}']
            predict_columns.append(f'prediction{i}')
            master_oof = pd.concat([master_oof, sub], axis=1)

            bool_cutomer_same = (master_oof['customer_id_sub0'] == master_oof[f'customer_id_sub{i}']).sum() == validation_length
            print('check_length')
            if bool_cutomer_same:
                print('OK')
            else:
                print('NG')
            master_oof = master_oof.drop([f'customer_id_sub{i}'], axis=1)
    master_oof = master_oof.drop([f'customer_id_sub0'], axis=1)
    return master_oof, predict_columns

def cust_blend(dt, predict_columns = [],W = [1,1,1]):
    #Global ensemble weights
    #W = [1.15,0.95,0.85]
    
    #Create a list of all model predictions
    REC = []
    for predict_column in predict_columns:
        REC.append(dt[predict_column].split())
    
    #Create a dictionary of items recommended. 
    #Assign a weight according the order of appearance and multiply by global weights
    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))
    
    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())
    
    # Return the top 12 itens only
    return ' '.join(res[:12])

own_file_name = 'exp001'
random.seed(0)
answer_df = pd.read_csv('../input/handm-submission-files/validation_answer.csv').sort_values('customer_id').reset_index(drop=True)[['customer_id', 'answer']]
answer_customers = answer_df['customer_id'].to_list()

#fix hiroi san oof
pathes = [
    '../input/handm-submission-files/byfone_Chris_agetop12_repeatuser_validation.csv', 
    '../input/handm-submission-files/exp13_LightSANs_week2_4_8_ensemble_validation.csv',
]
def fix_customers_get_fixed_pathes(answer_df, pathes):
    fixed_pathes = []
    for path in pathes:
        _df = pd.read_csv(path)
        mask = _df['customer_id'].isin(answer_customers)
        _df = _df[mask].reset_index(drop=True)
        _df.to_csv(os.path.basename(path), index=False)
        fixed_pathes.append(os.path.basename(path))
    return fixed_pathes

fixed_pathes = fix_customers_get_fixed_pathes(answer_df, pathes)

pathes = [
    '../input/fork-of-h-m-eda-rule-base-by-customer-age-make-oof/submission_validation.csv', 
    '../input/handm-submission-files/exp038_validation_pred.csv',
    '../input/handm-submission-files/h-m-pure-pytorch-baseline_exp010_submission_oof.csv'
]
pathes.extend(fixed_pathes)

params = list(itertools.product(
    np.linspace(0.1, 0.5, 5).tolist(), 
    np.linspace(0.6, 1.5, 5).tolist(),
    np.linspace(0.2, 1.0, 5).tolist(),
    np.linspace(0.4, 1.0, 5).tolist(),
    np.linspace(0.8, 1.5, 5).tolist()
    ))


random.shuffle(params)

master_oof, predict_columns = read_csvs_and_output_predict_columns(pathes)
answer_df['prediction'] = master_oof.apply(cust_blend, predict_columns = predict_columns, W = [0, 0, 1, 0, 0], axis=1)
best_score = 0

for param in tqdm(params):
    param = list(param)
    answer_df['prediction'] = master_oof.apply(cust_blend, predict_columns = predict_columns, W = param, axis=1)

    score = get_oof_score(answer_df)
    
    if best_score < score:
        print(f'best_score improve:{score:.6f}_param{param}')
        best_score = score
        best_param = param
print('finish!!')
print(f'best_score was {best_score} param {best_param}')
print(f'best_score was {best_score} param {best_param}', file=codecs.open(f'{own_file_name}_bestscore_param.txt', 'w', 'utf-8'))
